In [3]:
import xarray as xa

import numpy as np
import os
from scipy import interpolate
import pandas as pd

import matplotlib as mpl
from netCDF4 import num2date, date2num
import datetime

In [2]:
filein = '/Users/bell/Downloads/home/ecoraid/sigraid/salo/ssmi_ice/1989031_bcb.asc2'

In [97]:
fid = pd.read_csv(filein,delimiter='\s+',header=None,names=['lat','lon','iceconc'])

In [27]:
groups = fid.groupby(pd.cut(fid.lon,360))

In [90]:
for ind,lbin in enumerate(list(groups.groups.keys())):
    try:
        tg = groups.get_group(lbin)

        print(lbin.mid,tg[(tg>=15) & (tg!=1)].min().lat,tg[(tg>=15) & (tg!=1)].min().iceconc)
    except KeyError:
        pass

0.3075 89.51 nan
26.427500000000002 89.64 nan
63.9065 89.64 nan
89.8535 89.51 nan
104.26849999999999 89.33 nan
111.9565 89.12 nan
116.76150000000001 88.91 nan
119.6445 88.68 nan
121.56649999999999 88.46 nan
123.48849999999999 88.24 nan
125.41050000000001 88.01 nan
126.3715 87.78 nan
127.33250000000001 87.32 100.0
128.2935 86.87 100.0
129.2545 86.41 99.0
130.21550000000002 85.49 99.0
131.1765 84.11 99.0
132.1375 81.81 99.0
133.0985 76.33 95.0
134.0595 48.36 91.0
135.0205 46.35 91.0
135.98149999999998 46.34 55.0
136.9425 46.32 49.0
137.9035 46.28 15.0
138.8645 46.24 40.0
139.8255 46.19 19.0
140.7865 46.12 20.0
141.7475 46.05 15.0
142.70850000000002 45.94 20.0
143.6695 45.85 16.0
144.63049999999998 45.75 15.0
145.5915 45.61 15.0
146.5525 45.49 16.0
147.5135 45.32 16.0
148.4745 45.18 16.0
149.4355 44.99 15.0
150.3965 44.83 24.0
151.35750000000002 44.61 22.0
152.3185 44.38 16.0
153.27949999999998 44.2 22.0
154.2405 43.94 18.0
155.2015 43.67 15.0
156.1625 43.39 17.0
157.1235 43.09 16.0
158.0

In [100]:
fid.drop(fid.loc[fid['iceconc']<15].index, inplace=True)

In [101]:
fid.to_csv('/Users/bell/Downloads/home/ecoraid/sigraid/salo/ssmi_ice/1989031_bcb.iceonly.asc2')

In [102]:
fid

,lat,lon,iceconc
92,45.28,148.18,22
93,45.32,147.91,22
105,45.79,144.58,20
106,45.82,144.30,33
107,45.85,144.02,38
108,45.88,143.74,24
115,46.07,141.75,19
246,45.98,144.63,24
247,46.02,144.34,36
248,46.05,144.06,42
